In [1]:
%cd ../

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [5]:
import pandas as pd

In [2]:
from sklearn.decomposition import NMF

In [3]:
from price_predictor.trainers import train_linear_model, train_ensamble_model

In [6]:
pd.set_option('display.max_columns', 50)

In [7]:
data = pd.read_csv('data/cars.csv')
data.dropna(inplace=True)
data

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,body_type,has_warranty,state,drivetrain,price_usd,is_exchangeable,location_region,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,universal,False,owned,all,10900.00,False,Минская обл.,9,13,False,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,universal,False,owned,all,5000.00,True,Минская обл.,12,54,False,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,suv,False,owned,all,2800.00,True,Минская обл.,4,72,False,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,sedan,False,owned,all,9999.00,True,Минская обл.,9,42,True,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,universal,False,owned,all,2134.11,True,Гомельская обл.,14,7,False,True,False,True,True,False,False,False,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,Chrysler,300,automatic,silver,290000,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,2750.00,True,Минская обл.,5,85,False,True,False,False,True,True,False,False,True,True,301
38527,Chrysler,PT Cruiser,mechanical,blue,321000,2004,diesel,False,diesel,2.2,hatchback,False,owned,front,4800.00,True,Брестская обл.,4,20,False,True,False,False,True,True,False,False,True,True,317
38528,Chrysler,300,automatic,blue,777957,2000,gasoline,False,gasoline,3.5,sedan,False,owned,front,4300.00,False,Минская обл.,3,63,False,True,False,False,True,True,False,False,True,True,369
38529,Chrysler,PT Cruiser,mechanical,black,20000,2001,gasoline,False,gasoline,2.0,minivan,False,owned,front,4000.00,True,Брестская обл.,7,156,False,True,False,False,False,False,False,False,False,True,490


In [8]:
dummies = pd.get_dummies(data)
dummies

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed,manufacturer_name_Acura,manufacturer_name_Alfa Romeo,manufacturer_name_Audi,manufacturer_name_BMW,manufacturer_name_Buick,...,engine_type_gasoline,body_type_cabriolet,body_type_coupe,body_type_hatchback,body_type_liftback,body_type_limousine,body_type_minibus,body_type_minivan,body_type_pickup,body_type_sedan,body_type_suv,body_type_universal,body_type_van,state_emergency,state_new,state_owned,drivetrain_all,drivetrain_front,drivetrain_rear,location_region_Брестская обл.,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
0,190000,2010,False,2.5,False,10900.00,False,9,13,False,True,True,True,False,True,False,True,True,True,16,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
1,290000,2002,False,3.0,False,5000.00,True,12,54,False,True,False,False,True,True,False,False,False,True,83,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0
2,402000,2001,False,2.5,False,2800.00,True,4,72,False,True,False,False,False,False,False,False,True,True,151,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0
3,10000,1999,False,3.0,False,9999.00,True,9,42,True,False,False,False,False,False,False,False,False,False,86,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0
4,280000,2001,False,2.5,False,2134.11,True,14,7,False,True,False,True,True,False,False,False,False,True,7,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,False,True,False,False,True,True,False,False,True,True,301,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
38527,321000,2004,False,2.2,False,4800.00,True,4,20,False,True,False,False,True,True,False,False,True,True,317,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0
38528,777957,2000,False,3.5,False,4300.00,False,3,63,False,True,False,False,True,True,False,False,True,True,369,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
38529,20000,2001,False,2.0,False,4000.00,True,7,156,False,True,False,False,False,False,False,False,False,True,490,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0


In [16]:
dummies.lt(0).sum().sum()

0

In [17]:
dummies.describe()

,odometer_value,year_produced,engine_capacity,price_usd,number_of_photos,up_counter,duration_listed,manufacturer_name_Acura,manufacturer_name_Alfa Romeo,manufacturer_name_Audi,manufacturer_name_BMW,manufacturer_name_Buick,manufacturer_name_Cadillac,manufacturer_name_Chery,manufacturer_name_Chevrolet,manufacturer_name_Chrysler,manufacturer_name_Citroen,manufacturer_name_Dacia,manufacturer_name_Daewoo,manufacturer_name_Dodge,manufacturer_name_Fiat,manufacturer_name_Ford,manufacturer_name_Geely,manufacturer_name_Great Wall,manufacturer_name_Honda,...,engine_type_gasoline,body_type_cabriolet,body_type_coupe,body_type_hatchback,body_type_liftback,body_type_limousine,body_type_minibus,body_type_minivan,body_type_pickup,body_type_sedan,body_type_suv,body_type_universal,body_type_van,state_emergency,state_new,state_owned,drivetrain_all,drivetrain_front,drivetrain_rear,location_region_Брестская обл.,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
count,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,...,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000,38521.000000
mean,248910.074427,2002.940863,2.055161,6637.160884,9.648062,16.306586,80.584616,0.001713,0.005374,0.064069,0.067703,0.001220,0.001116,0.001506,0.011293,0.010644,0.040549,0.001532,0.005737,0.007710,0.021339,0.066613,0.001843,0.000935,0.020690,...,0.665793,0.001947,0.016926,0.198230,0.014278,0.000312,0.035539,0.093663,0.003349,0.337764,0.134057,0.142961,0.020976,0.009605,0.011370,0.979024,0.139846,0.720075,0.140079,0.077620,0.078061,0.081514,0.064510,0.628774,0.069521
std,136059.499858,8.064761,0.671178,6425.198996,6.092860,43.291786,112.838577,0.041358,0.073109,0.244879,0.251239,0.034909,0.033392,0.038774,0.105666,0.102618,0.197246,0.039107,0.075527,0.087469,0.144514,0.249354,0.042893,0.030557,0.142346,...,0.471719,0.044082,0.128995,0.398671,0.118636,0.017647,0.185140,0.291363,0.057773,0.472954,0.340718,0.350038,0.143304,0.097535,0.106026,0.143304,0.346831,0.448968,0.347074,0.267576,0.268272,0.273626,0.245663,0.483139,0.254341
min,0.000000,1942.000000,0.200000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,158000.000000,1998.000000,1.600000,2100.000000,5.000000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,250000.000000,2003.000000,2.000000,4800.000000,8.000000,5.000000,59.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000

In [18]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

In [20]:
pipeline = make_pipeline(MinMaxScaler(), NMF(n_components=30), RandomForestRegressor)
score = cross_val_score(pipeline, dummies.drop('price_usd', axis=1), dummies['price_usd'])
score

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ivanstefanov/.pyenv/versions/3.10.7/envs/used-cars/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ivanstefanov/.pyenv/versions/3.10.7/envs/used-cars/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
TypeError: BaseForest.fit() missing 1 required positional argument: 'y'
